<a href="https://colab.research.google.com/github/maxim1982/MIPT_magistratura/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22mosh_homework_01_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 28.0 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 


In [ ]:
import random, time, math

import numpy as np
import pandas as pd
import plotly.express as px

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate
from kaggle_environments.envs.rps.utils import get_score
from kaggle_environments.envs.rps.agents import agents

## Объявление глобальных переменных

In [ ]:
games = []
games_stat = {}
current_action, current_action_countdown = 0, 0
total_score, interim_score = 0, 0
shift = 0
shift_counter = 0

## Описания агентов

### Агент UNO - Выдаёт случайные значения, повторяя их случайное количество раз

In [ ]:
def mosh_agent_uno(observation, configuration):
    """
    Агент UNO
    Выдаёт случайные значения, повторяя их случайное количество раз
    
    current_action_countdown - счётчик количества, при достижении нуля, 
        заполняется случайным значением в диапазоне (min_repeat, max_repeat)
    current_action - действие (результат агента), повторяемое до обнуления 
        счётчика
    
    Гиперпараметры: 
    min_repeat, max_repeat - минимальное и максимальное число повторений
    """

    global games, current_action, current_action_countdown, total_score
    min_repeat, max_repeat = 1, 3

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        current_action_countdown = 0
        total_score = 0

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        games[-1][2] = get_score(games[-1][0], games[-1][1])
        total_score += games[-1][2]

    while current_action_countdown == 0:
        # если в гиперпараметрах кто-то установит min_repeat = 0
        current_action_countdown = random.randint(min_repeat, max_repeat)
        current_action = random.randint(0, 2)

    current_action_countdown -= 1
    # current_action = random.randint(0,2)
    games.append([current_action, None, 0])

    return current_action

### Агент DOS - Считает стратегию противника, основываясь прошлых результатах

In [ ]:
def mosh_agent_dos(observation, configuration):
    """
    Агент DOS
    Считает стратегию противника, основываясь на своём позапрошлом результате,
    и прошлом результате противника
    Вычисляет сдвиг (тактику) противника и старается его переиграть, 
    предвосхищая следующий ход противника на основании своего предыдущего хода
    и сдвига

    Гиперпараметры: 
    shift - сдвиг / изменение нашего хода
    """

    global games, current_action_countdown, total_score
    shift = 1

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        current_action_countdown = 0
        total_score = 0

    # если сыграли хотя бы одну игру
    if games:
        # в списке games сохраняем
        # [0] - наш ход
        # [1] - ход противника
        # [2] - результат игры
        # [3] - "тактический" сдвиг противника
        games[-1][1] = observation.lastOpponentAction
        games[-1][2] = get_score(games[-1][0], games[-1][1])
        total_score += games[-1][2]

    if len(games) > 1:
        # если сыграли больше одной игры, то можем проанализировать противника
        games[-1][3] = games[-1][1] - games[-2][0]
        current_action = (games[-1][0] + games[-1][3] + shift)% 3
    else:
        # если сыграно меньше двух, то выдаём случайное значение
        current_action = random.randint(0,2)

    games.append([current_action, None, None, None])

    return current_action


### Агент TRES - Выдаёт случайный результат с некоторым уклоном

In [ ]:
def mosh_agent_tres(observation, configuration):
    """
    Агент TRES
    Выдаёт случайный результат с уклоном в сторону какого-либо действия

    Гиперпараметры: 
    priority_action - приоритезируемое действие
    priority_power - сила приоритета (вес действия, добавляемый в список действий)
    """

    # rock = 0
    # paper = 1
    # scissors = 2

    # приоритезируем ножницы с мощностью 3
    priority_action = 2
    priority_power = 3

    # определяем список возможных действий
    actions = [0, 1, 2] + [priority_action]*priority_power

    return actions[random.randint(0,len(actions)-1)]
 

### Агент CUATRO - действует в зависимости от победы/поражения на прошлом шагу

In [ ]:
def mosh_agent_cuatro(observation, configuration):
    """
    Агент CUATRO
    Если на предыдущем ходу мы выиграли, то меняем своё текущее действие на -1
    Если на предыдущем ходу мы проиграли, то меняем своё текущее действие на +1
    Кроме этого, мы делаем дополнительный сдвиг, который меняем, 
        если начинаем проигрывать
    Если была ничья, то делаем случайный ход

    Гиперпараметры: 
    shift_delay - через сколько ходов меняем сдвиг (shift)
    """

    global games, current_action_countdown, \
        total_score, interim_score, \
        shift, shift_counter

    shift_delay = 10

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        current_action_countdown = 0
        total_score, interim_score = 0, 0
        shift, shift_counter = 0, shift_delay
        
        last_score = 0

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score
        interim_score += last_score

    if last_score == 0:
        current_action = random.randint(0,2)
    else:
        current_action = (games[-1][0] - last_score + shift) % 3

    if interim_score <= 0:
        interim_score = 0
        shift_counter = shift_delay
        shift += 1

    shift_counter -= 1

    games.append([current_action, None, None])
    return current_action
    

### Агент CUATRO - меняет тактику в зависимости от результата серии игр

In [ ]:
def mosh_agent_cinco(observation, configuration):
    """
    Агент CINCO
    Если на предыдущем ходу мы выиграли, то повторяем
    Если на предыдущем ходу мы проиграли, то делаем сдвиг относительно противника
    Если была ничья, то повторяем ход
    Если сильно проигрываем, то меняем сдвиг

    Гиперпараметры: 
    shift_delay - через сколько ходов меняем сдвиг (shift)
    """
    
    global games, current_action_countdown, \
        total_score, interim_score, \
        shift, shift_counter
    
    shift_delay = 10

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        current_action_countdown = 0
        total_score, interim_score = 0, 0
        shift, shift_counter = 0, shift_delay
        
        current_action = random.randint(0,2)

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

        if last_score == 0:
            current_action = random.randint(0,2)
        elif last_score > 0:
            current_action = games[-1][0]
        else:
            current_action = (games[-1][1] + shift) % 3

    if interim_score <= 0:
        interim_score = 0
        shift_counter = shift_delay
        shift += 1

    shift_counter -= 1

    games.append([current_action, None, None])
    return current_action

### Агент SEIS - использует статистику побед на основании двух предыдущих игр

In [ ]:
def mosh_agent_seis(observation, configuration):
    """
    Агент SEIS
    Анализирует два последних хода и их результаты 
    Сохраняет выигрышные и проигрышные варианты
    На основании статистики, выбираем наилучший ход

    Гиперпараметры:
    нет
    """
    global games, games_stat, total_score

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        games_stat = {}
        total_score = 0
    
    if observation.step < 2:
        current_action = random.randint(0,2)

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

        combination = (games[-2][0], games[-2][1], games[-1][0])
        combination_score = games_stat.get(combination, 0)
        games_stat[combination] = combination_score + last_score
        best_combination = ()
        best_combination_score = 0
        for i in range(3):
            combination_candidate = (games[-1][0], games[-1][1], i)
            combination_candidate_score = games_stat.get(combination_candidate, 0)
            if combination_candidate_score > best_combination_score:
                best_combination = combination_candidate
                best_combination_score = combination_candidate_score
        if best_combination_score <= 0:
            best_combination = (0, 0, random.randint(0,2))

        current_action = best_combination[-1]
            
    games.append([current_action, None, None])
    return current_action    


### Агент SIETE - повторяет свой либо чужой ход в зависимости от результата

In [ ]:
def mosh_agent_seite(observation, configuration):
    """
    Агент SIETE
    Если выиграли на предыдущем шаге, то повторяем свой предыдущий ход
    Если проиграли, то повторяем ход противника
    Если ничья, то выдаём результат, отличный от предыдущего хода

    Гиперпараметры: 
    Нет
    """

    global games, total_score
    
    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        total_score = 0
        current_action = random.randint(0,2)

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

        if last_score > 0:
            current_action = games[-1][0]
        elif last_score < 0:
            current_action = games[-1][1]
        else:
            actions = [x for x in range(3) if x != games[-1][0]]
            current_action = actions[random.randint(0,1)]

    games.append([current_action, None, None])
    return current_action

### Агент OCHO - выдаёт случайные результаты из двух вариантов вместо трёх


In [ ]:
def mosh_agent_ocho(observation, configuration):
    """
    Агент OCHO
    Выдаёт случайные результаты из двух вариантов вместо трёх
    Из возможных действий исключается одно

    Гиперпараметры: 
    action_exclude - исключаемое действие
    """
    # rock = 0
    # paper = 1
    # scissors = 2

    action_exclude = 1
    
    actions = [0, 1, 2]
    actions.remove(action_exclude)
    
    current_action = actions[random.randint(0,1)]

    return current_action

### Агент NUEVE - раз в несколько ходов повторяет приоритетное действие

In [ ]:
def mosh_agent_nueve(observation, configuration):
    """
    Агент NUEVE
    Приоритетное действие повторяется раз в несколько ходов
    Во всех остальных случаях действие выбирается с учётом весов каждого шага
    
    Гиперпараметры: 
    priority_action - приоритетное действие
    cadence - ритм, с которым повторяется приоритетное действие
    rock_weight - вес камня
    paper_weight - вес бумаги
    scissors_weight - вес ножниц
    """
    
    global games, total_score, current_action_countdown

    priority_action = 2
    cadence = 3
    rock_weight = 4
    paper_weight = 3
    scissors_weight = 2

    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        total_score = 0
        current_action_countdown = 0
    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

    if current_action_countdown == 0:
        current_action = priority_action
        current_action_countdown = cadence
    else:
        actions = [0]*rock_weight + [1]*paper_weight + [2]*scissors_weight
        current_action = actions[random.randint(0, len(actions)-1)]
        current_action_countdown -= 1

    games.append([current_action, None, None])
    return current_action

### Агент DIEZ - Анализируем статистику игр по своим последних шагам

In [ ]:
def mosh_agent_diez(observation, configuration):
    """
    Агент DIEZ
    Анализируем статистику по своим последних действиям
    Выбираем наиболее благоприятный статистически шаг
    
    Гиперпараметры: 
    analyze_range - количество последних действий для анализа
    """
    
    global games, games_stat, total_score

    analyze_range = 3
    
    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        games_stat = {}
        total_score = 0

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

    if observation.step < analyze_range:
        current_action = random.randint(0,2)

    else:
        combination = [
            games[x][0] for x in range(-analyze_range, 0)
            ]
        combination = tuple(combination)
        combination_score = games_stat.get(combination, 0)
        games_stat[combination] = combination_score + last_score
        
        best_combination = ()
        best_combination_score = 0
        for i in range(3):
            combination_candidate = [
                games[x][0] for x in range(-analyze_range+1, 0)
                ]
            combination_candidate.append(i)
            combination_candidate = tuple(combination_candidate)
            combination_candidate_score = games_stat.get(combination_candidate, 0)
            if combination_candidate_score > best_combination_score:
                best_combination = combination_candidate
                best_combination_score = combination_candidate_score
        if best_combination_score <= 0:
            current_action = random.randint(0,2)
        else:
            current_action = best_combination[-1]

    games.append([current_action, None, None])
    return current_action

### Агент ONCE - Меняет последовательность действий, если проигрывает

In [ ]:
def mosh_agent_once(observation, configuration):
    """
    Агент ONCE
    Генерирует случайную последовательность наблюдает за результатом работы
    Если выбранная комбинация проигрывает, то меняет её

    Гиперпараметры: 
    min_combination, max_combination - мин. и макс. длина комбинации
    """

    def get_combination(min_combination, max_combination):
        combination = []
        combination_len = random.randint(min_combination, max_combination)
        while len(combination) < combination_len:
            combination.append(random.randint(0,2))
        
        return combination

    
    global games, total_score, interim_score, \
        combination, combination_step 

    min_combination, max_combination = 4, 8
    
    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        total_score = 0
        interim_score = 0
        
    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score
        interim_score += last_score
    
    if interim_score <= 0:
        combination = get_combination(min_combination, max_combination)
        combination_step = 0

    current_action = combination[combination_step]
    combination_step += 1
    combination_step %= len(combination)

    games.append([current_action, None, None])
    return current_action

### Агент DOCE - выдаёт ход в зависимости от текущего времени

In [ ]:
def mosh_agent_doce(observation, configuration):
    """
    Агент DOCE
    Определяет остаток по модулю 3 от текущего времени
    Этот результат используется в качестве действия
    
    Гиперпараметры: 
    Нет
    """
    global games, total_score
    
    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        total_score = 0

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

    current_action = math.floor(time.time()*1000_000 % 3)

    games.append([current_action, None, None])
    return current_action    


In [ ]:
def mosh_agent_zero(observation, configuration):
    """
    Агент ZERO
    
    Гиперпараметры: 
    Нет
    """

    global games, total_score
    
    if observation.step == 0:
        # инициализируем переменные в начале игры
        games = []
        total_score = 0

    else:
        # сохраняем предыдущий результат противника
        games[-1][1] = observation.lastOpponentAction
        last_score = int(get_score(games[-1][0], games[-1][1]))
        games[-1][2] = last_score
        total_score += last_score

    current_action = random.randint(0,2)

    games.append([current_action, None, None])
    return current_action

In [ ]:
result = evaluate(
    "rps", 
    [mosh_agent_uno, "counter_reactionary"], 
    configuration={"episodeSteps": 200}
)

result, total_score, games[:15]

([[-53.0, 53.0]],
 -52.0,
 [[1, 1, 0],
  [2, 2, 0],
  [2, 0, -1.0],
  [2, 2, 0],
  [2, 0, -1.0],
  [0, 2, 1.0],
  [0, 1, -1.0],
  [0, 0, 0],
  [0, 1, -1.0],
  [0, 0, 0],
  [2, 1, 1.0],
  [2, 0, -1.0],
  [1, 2, -1.0],
  [1, 1, 0],
  [1, 2, -1.0]])

## Турнир агентов

### Проведение турнира

In [ ]:
mosh_agents = {
    "mosh_agent_uno": mosh_agent_uno,
    "mosh_agent_dos": mosh_agent_dos,
    "mosh_agent_tres": mosh_agent_tres,
    "mosh_agent_cuatro": mosh_agent_cuatro,
    "mosh_agent_cinco": mosh_agent_cinco,
    "mosh_agent_seis": mosh_agent_seis,
    "mosh_agent_seite": mosh_agent_seite,
    "mosh_agent_ocho": mosh_agent_ocho,
    "mosh_agent_nueve": mosh_agent_nueve,
    "mosh_agent_diez": mosh_agent_diez,
    "mosh_agent_once": mosh_agent_once,
    "mosh_agent_doce": mosh_agent_doce,
}

# Количество раундов, которые сыграют агенты друг с другом
rounds = 3

# Количество игр в каждом раунде
episode_steps = 50

# Минимальное количество очков для победы в раунде
tie_reward_threshold = 10

# Если хотим видеть результат каждого поединка онлайн
verbose = True

# Результаты всех игр сохраняем для дальнейшего анализа
df = pd.DataFrame(columns = ["mosh_strategy", "opponent_strategy", "score"])

for r in range(rounds):
    for mosh_agent_name, mosh_agent_fnc in mosh_agents.items():
        for agent in agents:
            result = evaluate(
                "rps", 
                [mosh_agent_fnc, agent], 
                configuration={
                    "episodeSteps": episode_steps,
                    "tieRewardThreshold": tie_reward_threshold,
                    }
            )
            mosh_score = result[0][0]
            agent_score = result[0][1]
            game_score = mosh_score - agent_score
            if game_score > 0:
                winner = mosh_agent_name
            elif game_score < 0:
                winner = agent
            else:
                winner = "draw"
            if verbose:
                print(f"round: {r}: {mosh_agent_name} vs {agent} = {mosh_score}:{agent_score} | Winner - {winner}")

            df = df.append({
                    "mosh_strategy": mosh_agent_name, 
                    "opponent_strategy": agent, 
                    "score": game_score,
                }, ignore_index=True)


round: 0: mosh_agent_uno vs rock = 0:0 | Winner - draw
round: 0: mosh_agent_uno vs paper = 14.0:-14.0 | Winner - mosh_agent_uno
round: 0: mosh_agent_uno vs scissors = -10.0:10.0 | Winner - scissors
round: 0: mosh_agent_uno vs copy_opponent = 0:0 | Winner - draw
round: 0: mosh_agent_uno vs reactionary = -20.0:20.0 | Winner - reactionary
round: 0: mosh_agent_uno vs counter_reactionary = -14.0:14.0 | Winner - counter_reactionary
round: 0: mosh_agent_uno vs statistical = -13.0:13.0 | Winner - statistical
round: 0: mosh_agent_dos vs rock = -25.0:25.0 | Winner - rock
round: 0: mosh_agent_dos vs paper = 0:0 | Winner - draw
round: 0: mosh_agent_dos vs scissors = 0:0 | Winner - draw
round: 0: mosh_agent_dos vs copy_opponent = 48.0:-48.0 | Winner - mosh_agent_dos
round: 0: mosh_agent_dos vs reactionary = 46.0:-46.0 | Winner - mosh_agent_dos
round: 0: mosh_agent_dos vs counter_reactionary = 48.0:-48.0 | Winner - mosh_agent_dos
round: 0: mosh_agent_dos vs statistical = 12.0:-12.0 | Winner - mosh_a

### Вывод результатов турнира

In [ ]:
# Расставляем результаты поединков
df.score = pd.to_numeric(df.score)
df["win"] = df.apply(lambda row: 1 if row["score"] > 0 else 0, axis=1)
df["lose"] = df.apply(lambda row: 1 if row["score"] < 0 else 0, axis=1)
df["draw"] = df.apply(lambda row: 1 if row["score"] == 0 else 0, axis=1)

**Попарное сравнение агентов**

In [ ]:
# Визуализируем результаты
fig = px.density_heatmap(df, x="opponent_strategy", y="mosh_strategy", z="score")
fig.show()

**Таблица лидеров**

In [ ]:
# Напечатаем турнирную таблицу - собственные агенты
df.groupby("mosh_strategy").sum().sort_values(["win", "score"], ascending=(False))

,score,win,lose,draw
mosh_strategy,,,,
mosh_agent_seis,1336,20,0,1
mosh_agent_diez,892,15,0,6
mosh_agent_seite,964,14,0,7
mosh_agent_dos,972,13,4,4
mosh_agent_cuatro,806,12,4,5
mosh_agent_cinco,764,11,4,6
mosh_agent_once,50,3,3,15
mosh_agent_ocho,-172,3,9,9
mosh_agent_tres,-202,3,7,11


In [ ]:
# Напечатаем турнирную таблицу - чужеродные агенты
df.groupby("opponent_strategy").sum().sort_values(["win", "score"], ascending=(False))

,score,win,lose,draw
opponent_strategy,,,,
paper,1420,19,2,15
scissors,1404,19,2,15
rock,926,17,8,11
copy_opponent,1018,15,1,20
reactionary,342,10,8,18
counter_reactionary,370,8,6,22
statistical,-316,8,14,14
